In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.datasets.mnist import load_data
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input,Dense,Conv2D,MaxPooling2D,Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.vgg16 import VGG16


In [2]:
(X_train,Y_train),(X_test,Y_test) = load_data()

In [3]:
len(X_train)

60000

In [4]:
X_train, X_val, Y_train, Y_val = train_test_split(X_train,Y_train, test_size = 0.16666, random_state = 42)


In [13]:
len(X_train)

50000

# Define Model

In [26]:
def create_model():
    vgg16_model = VGG16(weights = 'imagenet', include_top= False, input_shape= (32,32,3))
    vgg16_model.trainable = False
    inputs = vgg16_model.input
    outputs = vgg16_model.output
    x = Flatten()(outputs)
    x =Dense(512,activation = 'relu')(x)
    outputs =Dense(10, activation ='softmax')(x)

    model = Model(inputs,outputs)
    return model
    
    
    

In [27]:
model_1 =create_model()


In [28]:
model_1.summary(show_trainable = True)


Model: "model"
____________________________________________________________________________
 Layer (type)                Output Shape              Param #   Trainable  
 input_3 (InputLayer)        [(None, 32, 32, 3)]       0         N          
                                                                            
 block1_conv1 (Conv2D)       (None, 32, 32, 64)        1792      N          
                                                                            
 block1_conv2 (Conv2D)       (None, 32, 32, 64)        36928     N          
                                                                            
 block1_pool (MaxPooling2D)  (None, 16, 16, 64)        0         N          
                                                                            
 block2_conv1 (Conv2D)       (None, 16, 16, 128)       73856     N          
                                                                            
 block2_conv2 (Conv2D)       (None, 16, 16, 128)       147584

In [35]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, array_to_img
x_train = tf.expand_dims(X_train, axis=-1)
x_test = tf.expand_dims(X_test, axis=-1)
x_val  = tf.expand_dims(X_val, axis = -1)

x_train = tf.image.grayscale_to_rgb(x_train)
x_test = tf.image.grayscale_to_rgb(x_test)
x_val  = tf.image.grayscale_to_rgb(x_val)

x_train = tf.image.resize(x_train, [32, 32])
x_test = tf.image.resize(x_test, [32, 32])
x_val  =tf.image.resize(x_val,[32,32])
print(x_train.shape)
print(x_test.shape)
print(x_val.shape)

(50000, 32, 32, 3)
(10000, 32, 32, 3)
(10000, 32, 32, 3)


In [38]:
x_train = x_train/255.0
x_test  = x_test/255.0
x_val   = x_val/255.0

In [39]:
y_train = to_categorical(Y_train,10)
y_test  = to_categorical(Y_test,10)
y_val   = to_categorical(Y_val,10)

In [40]:
model_1.compile(loss = 'categorical_crossentropy',optimizer ='adam',metrics =['accuracy'])

In [ ]:
history_1 = model_1.fit(x_train, y_train, validation_data =(x_val,y_val),epochs = 10)

Epoch 1/10


1563/1563 [==============================] - 413s 262ms/step - loss: 0.2837 - accuracy: 0.9142 - val_loss: 0.2083 - val_accuracy: 0.9280
Epoch 2/10
1563/1563 [==============================] - 384s 246ms/step - loss: 0.1380 - accuracy: 0.9554 - val_loss: 0.1763 - val_accuracy: 0.9430
Epoch 3/10
1563/1563 [==============================] - 359s 230ms/step - loss: 0.1160 - accuracy: 0.9618 - val_loss: 0.1206 - val_accuracy: 0.9614
Epoch 4/10
1563/1563 [==============================] - 357s 228ms/step - loss: 0.1017 - accuracy: 0.9669 - val_loss: 0.1066 - val_accuracy: 0.9659
Epoch 5/10
1563/1563 [==============================] - 356s 228ms/step - loss: 0.0917 - accuracy: 0.9698 - val_loss: 0.1091 - val_accuracy: 0.9643
Epoch 6/10
1563/1563 [==============================] - ETA: 0s - loss: 0.0849 - accuracy: 0.9718

# Define data augmentation

In [ ]:
def augmentation(X,Y):
    datagen = ImageDataGenerator(
        rotation_range = 40,
        shear_range = 0.2,
        zoom_range = .2,
        width_shift_range=0.15,
        height_shift_range=0.15
    )

    augmented_images = []
    augmented_labels = []

    batch_size = len(X)
    generator = datagen.flow(X,Y,batch_size = batch_size)
    images ,labels = next(generator)
    
    augmented_images.extend(images)
    augmented_labels.extend(labels)

    augmented_images = np.array(augmented_images)
    augmented_labels = np.array(augmented_labels)

    
    x_combined = np.concatenate((X,augmented_images))
    y_combined = np.concatenate((Y,augmented_labels))
    return (x_combined,y_combined)
    
    

In [ ]:
(x_train_augmented, y_train_augmented) = augmentation(x_train,y_train)
(x_test_augmented, y_test_augmented) =augmentation(x_test,y_test)



In [ ]:
len(y_test_augmented)

# Create model using augmented dataset

In [ ]:
augmented_model = create_model()
augmented_model.summary()

# Compile and fit the model

In [ ]:
augmented_model.compile(loss = 'categorical_crossentropy', optimizer ='adam', metrics = ['accuracy'])
history2 = augmented_model.fit(x_train_augmented, y_train_augmented, validation_data =(X_val,Y_val), epochs = 10)

In [ ]:
test_acc1 = model.evaluate(x_test, x_test)
test_acc2 = model.evaluate(x_test_augmented,y_test_augmented)

In [ ]:
test_acc2 = augmented_model.evaluate(X_test,Y_test)
test_acc3 = augmented_model.evaluate(x_test_augmented,y_test_augmented)


#  Ploting accuracy and lost model_1

In [ ]:
history = history_1.history
plt.figure(figsize = (12,4))
plt.subplot(1,2,1)
plt.plot(history['accuracy'])
plt.plot(history['val_accuracy'])
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(['train_accuracy','validation_accuracy'])

plt.subplot(1,2,2)
plt.plot(history['loss'])
plt.plot(history['val_loss'])
plt.legend(['training_loss','validation_loss'])
plt.show()

# Ploting accuracy and loss augmented model

In [ ]:
history = history2.history
plt.figure(figsize = (12,4))
plt.subplot(1,2,1)
plt.plot(history['accuracy'])
plt.plot(history['val_accuracy'])
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(['train_accuracy','validation_accuracy'])

plt.subplot(1,2,2)
plt.plot(history['loss'])
plt.plot(history['val_loss'])
plt.legend(['training_loss','validation_loss'])
plt.show()